In [1]:
import os
import glob
from pathlib import Path

examination_paper_list= []

path = Path("./answer_markdown")

for file in path.glob("*.md"):
    with open(file, "r", encoding="utf-8") as f:
        examination_paper_list.append({
            "file_path": file,
            "text": f.read()
        })

In [33]:
def per_process(text):
    liens = text.splitlines()
    liens = map(lambda x: x.replace("> ","").replace(">","").replace("*", ""), liens)
    liens = filter(lambda x: x.strip() != "", liens)
   
    return "\n".join(liens)

In [372]:
import re

def extract_chinese_number(paragraph):
    pattern = r'^[一二三四五六七八九十][、．.]'
    match = re.match(pattern, paragraph)
    if match:
        return match.group(0)[0]
    else:
        return None


def convert_chinese_uppercase_to_number(chinese_number):
    conversion_table = {
        '一': 1, '二': 2, '三': 3, '四': 4, '五': 5,
        '六': 6, '七': 7, '八': 8, '九': 9, '十': 10
    }
    return conversion_table[chinese_number]


def get_topic_type_with_index(text):
    lines = text.splitlines()
    
    topic_type_with_index = {}
    
    for index, line in enumerate(lines):
        if extract_chinese_number(line):
            exists = False
            
            for _index in topic_type_with_index:
                if topic_type_with_index[_index] == line:
                    exists = True
                    break
                    
            if not exists:
                topic_type_with_index[index] = line
   
    return topic_type_with_index
    

def get_option_start_and_end_index(topic_type_with_index, lines):
    start_index = 0
    end_index = max([index for index in topic_type_with_index])
    
    for index in topic_type_with_index:
        if start_index:
            end_index = index
            break
            
        if "选择题" in topic_type_with_index[index]:
            start_index = index
    
    last_option_index = end_index
  
    for index, line in enumerate(lines[end_index-1::-1]):
        if "D．" in line or "（D）" in line:
            end_index = last_option_index - index
            break
            
    return start_index, end_index

def find_next_answer_index(lines, current_index, end_index):
    next_answer_index = end_index
    
    for index in range(current_index+1, end_index-1):
        match = re.match(r'^(\d+)[．.|\(]', lines[index])
        if not match:
             match = re.match(r'^\（\d+）', lines[index])
                
        if match or extract_chinese_number(lines[index]):
            next_answer_index = index
            break
            
    return next_answer_index
    
def find_next_option_index(lines, current_index):
    next_option_index = 0
    for index in range(current_index+1, current_index+10):
        
        if "A．" in lines[index] or "（A）" in lines[index]:
            next_option_index = index
            break
            
    return next_option_index

def get_all_option_answer_and_options(text):
    lines = text.splitlines()
    topic_type_with_index = get_topic_type_with_index(text)
    
    if not topic_type_with_index:
        print("无法分界，选择题、填空题等")
        return
    
    start_index, end_index = get_option_start_and_end_index(topic_type_with_index, lines)
    
    answer_and_options = {}
    for index in range(start_index + 1, end_index):
        match = re.match(r'^(\d+)[．.]', lines[index])
        if not match:
            match = re.match(r'^\（\d+）', lines[index])
  
        if match:
#             print(f"index {index}")
            next_answer_index = find_next_answer_index(lines, index, end_index)
            next_option_index = find_next_option_index(lines, index)
            if next_option_index >= next_answer_index:
                next_option_index = next_answer_index
#             print(f"next_answer_index {next_answer_index}")
#             print(f"next_option_index {next_option_index}")
#             print( "\n".join(lines[index: next_option_index]))
#             print()
            answer_and_options[index] = {
                "topic" : "\n".join(lines[index: next_option_index]),
                "option": " ".join(lines[next_option_index: next_answer_index])
            }
            
#             break

    return answer_and_options

# 测试算法

In [393]:
test_text = examination_paper_list[40]["text"]

In [394]:
per_process(test_text)

'2004年普通高等学校招生全国统一考试\n数学（文科类）（湖北卷）\n一、选择题：本大题共12小题，每小题5分，共60分，在每小题给出的四个选项中，只有一项是符合题目要求的.\n1．设等于 （ ）\nA．{1,4} B．{1,6} C．{4,6} D．{1,4,6}\n2．已知点M（6，2）和M~2~（1，7）.直线y=mx---7与线段M~1~M~2~的交点M分有向线段M~1~M~2~的比为3：2，则m的值为 （ ）\nA． B． C． D．4\n3．已知函数的解析式可能为 （ ）\nA． B．\nC． D．\n4．两个圆的公切线有且仅有 （ ）\nA．1条 B．2条 C．3条 D．4条\n5．若函数、三、四象限，则一定有（ ）\nA． B．\nC． D．\n6．四面体ABCD四个面的重心分别为E、F、G、H，则四面体EFGH的表面积与四面体ABCD的表面积的比值是 （ ）\nA． B． C． D．\n7．已知为非零的平面向量. 甲： （ ）\nA．甲是乙的充分条件但不是必要条件\nB．甲是乙的必要条件但不是充分条件\nC．甲是乙的充要条件\nD．甲既不是乙的充分条件也不是乙的必要条件\n8．已知有 （ ）\nA．最大值 B．最小值 C．最大值1 D．最小值1\n9．已知数列{}的前n项和其中a、b是非零常数，则存在数列{}、{}使得 （ ）\nA．为等差数列，{}为等比数列\nB．和{}都为等差数列\nC．为等差数列，{}都为等比数列\nD．和{}都为等比数列\n10．若则下列结论中不正确的是 （ ）\nA． B．\nC． D．\n11．将标号为1，2，...，10的10个球放入标号为1，2，...，10的10个盒子里，每个盒内放一个球，恰好3个球的标号与其在盒子的标号不一致的放入方法种数为（ ）\nA．120 B．240 C．360 D．720\n12．设是某港口水的深度y（米）关于时间t（时）的函数，其中.下表是该港口某一天从0时至24时记录的时间t与水深y的关系：\n  --- ---- ------ ------ ----- ------ ------ ------ ----- ------\n  t   0    3      6      9     12     15     18     21    24\n  y   12   15.1   

In [353]:
get_all_option_answer_and_options(per_process(test_text))

{3: {'topic': '1．函数 的定义域为 （ ）', 'option': 'A． B． C． D．'},
 5: {'topic': '2．设直线 ax+by+c=0的倾斜角为，且sin+cos=0，则a,b满足 （ ）',
  'option': 'A． B． C． D．'},
 7: {'topic': '3．设是函数f(x)=的反函数，则下列不等式中恒成立的是 （ ）', 'option': 'A． B． C． D．'},
 10: {'topic': '4．如果双曲线上一点P到右焦点的距离为, 那么点P到右准线的距离是（ ）',
  'option': 'A． B．13 C．5 D．'},
 12: {'topic': '5．把正方形ABCD沿对角线AC折起，当A、B C、D四点为顶点的三棱锥体积最大时，直线BD与平面ABC所成的角的大小为 （ ）',
  'option': 'A．90° B．60° C．45° D．30°'},
 14: {'topic': '6．某公司甲、乙、丙、丁四个地区分别有150 个、120个、180个、150个销售点.公司为了调查产品的情况，需从这600个销售点中抽取一个容量为100的样本，记这项调查为①；在丙地区中有20个特大型销售点，要从中抽取7个调查其收入和售后服务等情况，记这项调查为②.则完成这两项调查宜采用的抽样方法依次为 （ ）',
  'option': 'A．分层抽样法，系统抽样法 B．分层抽样法，简单随机抽样法 C．系统抽样法，分层抽样法 D．简单随机抽样法，分层抽样法'},
 17: {'topic': '7．若f(x)=-x^2^+2ax与在区间\\[1,2\\]上都是减函数，则a的值范围是 （ ）',
  'option': 'A． B． C．（0，1） D．'},
 19: {'topic': '8．已知向量,向量则的最大值，最小值分别是（ ）', 'option': 'A． B． C．16，0 D．4，0'},
 21: {'topic': '9．若函数f(x)=x^2^+bx+c的图象的顶点在第四象限，则函数f ^/^(x)的图象是 （ ）',
  'option': ''},
 22: {'topic': '10．从正方体的八个顶点中任取三个点作为三角形，直角三角形的个数为 （ ）',


In [330]:
all_topic_type = get_topic_type_with_index(per_process(test_text))
start_index, end_index = get_option_start_and_end_index(all_topic_type, per_process(test_text).splitlines())
start_index, end_index

(12, 43)

In [332]:
all_topic_type

{12: '一、选择题：本大题共12小题，每小题5分，共60分.',
 44: '二、填空题：本大题共4小题，每小题4分，共16分.把答案填在题中横线上.',
 52: '三、解答题：本大题共6小题，共74分.解答应写出文字说明，证明过程或演算步骤.',
 75: '一、选择题',
 79: '三、解答题'}

In [327]:
per_process(test_text).splitlines()

['2004年普通高等学校招生全国统一考试',
 '数学 (理工农林医类)',
 '本试卷分第Ⅰ卷（选择题）和第Ⅱ卷（非选择题）两部分。第Ⅰ卷1至1页，第Ⅱ卷3至10页。考试结束后，将试卷和答题卡一并交回。',
 '第Ⅰ卷',
 '注意事项：',
 '1．答第Ⅰ卷前，考生务必将自己的姓名、准考证号、考试科目涂写在答题卡上。',
 '2．每小题选出答案后，用铅笔在答题卡上对应题目的答案涂黑。如需改动，用橡皮擦干净后，再选涂其它答案标号。不能答在试题卷上。',
 '3．本卷共12小题，每小题5分，共60分。在每小题给出的四个选项中，只有一项是符合题目要求的。',
 '参考公式：',
 '三角函数的和差化积公式',
 '一、选择题',
 '1．设集合，，则集合中元素的个数为 （ ）',
 'A．1 B．2 C．3 D．4',
 '2．函数的最小正周期是 （ ）',
 'A． B． C． D．',
 '3．设数列是等差数列，且，是数列的前项和，则 （ ）',
 'A． B． C． D．',
 '4．圆在点处的切线方程为 （ ）',
 'A． B．',
 'C． D．',
 '5．函数的定义域为 （ ）',
 'A． B．',
 'C． D．',
 '6．设复数的辐角的主值为，虚部为，则= （ ）',
 'A． B． C． D．',
 '7．设双曲线的焦点在轴上，两条渐近线为，则该双曲线的离心率（ ）',
 'A． B． C． D．',
 '8．不等式的解集为 （ ）',
 'A． B． C． D．',
 '9．正三棱锥的底面边长为2，侧面均为直角三角形，则此三棱锥的体积为 （ ）',
 'A． B． C． D．',
 '10．在△ABC中，AB=3，BC=，AC=4，则边AC上的高为 （ ）',
 'A． B． C． D．',
 '11．设函数 ，则使得的自变量的取值范围为（ ）',
 'A． B．',
 'C． D．',
 '12．将4名教师分配到3所中学任教，每所中学至少1名，则不同的分配方案共有（ ）',
 'A．12种 B．24种 C．36种 D．48种',
 '第Ⅱ卷',
 '二、填空题（每小题4分，共16分.把答案填在题中横线上，解答应写出文字说明，证明过程或演算步骤.）',
 '13．用平面截半径为的球，如果球心到平面的距离为，那么截得小圆的面积与球的表面积的

# 统计一共可以提取多少个

In [379]:
loss = []

for examination_paper in examination_paper_list:
    text = examination_paper["text"]
    text = per_process(text)
    try:
        answer_and_options = get_all_option_answer_and_options(text)
        if not answer_and_options:
            loss.append(examination_paper)
    except:
        loss.append(examination_paper)

无法分界，选择题、填空题等
无法分界，选择题、填空题等
无法分界，选择题、填空题等
无法分界，选择题、填空题等
无法分界，选择题、填空题等
无法分界，选择题、填空题等
无法分界，选择题、填空题等
无法分界，选择题、填空题等
无法分界，选择题、填空题等
无法分界，选择题、填空题等
无法分界，选择题、填空题等
无法分界，选择题、填空题等
无法分界，选择题、填空题等
无法分界，选择题、填空题等
无法分界，选择题、填空题等
无法分界，选择题、填空题等
无法分界，选择题、填空题等
无法分界，选择题、填空题等
无法分界，选择题、填空题等
无法分界，选择题、填空题等
无法分界，选择题、填空题等
无法分界，选择题、填空题等
无法分界，选择题、填空题等
无法分界，选择题、填空题等
无法分界，选择题、填空题等
无法分界，选择题、填空题等
无法分界，选择题、填空题等
无法分界，选择题、填空题等
无法分界，选择题、填空题等
无法分界，选择题、填空题等
无法分界，选择题、填空题等
无法分界，选择题、填空题等
无法分界，选择题、填空题等
无法分界，选择题、填空题等
无法分界，选择题、填空题等
无法分界，选择题、填空题等
无法分界，选择题、填空题等
无法分界，选择题、填空题等
无法分界，选择题、填空题等
无法分界，选择题、填空题等
无法分界，选择题、填空题等
无法分界，选择题、填空题等
无法分界，选择题、填空题等
无法分界，选择题、填空题等
无法分界，选择题、填空题等
无法分界，选择题、填空题等
无法分界，选择题、填空题等
无法分界，选择题、填空题等
无法分界，选择题、填空题等
无法分界，选择题、填空题等
无法分界，选择题、填空题等
无法分界，选择题、填空题等
无法分界，选择题、填空题等
无法分界，选择题、填空题等
无法分界，选择题、填空题等
无法分界，选择题、填空题等
无法分界，选择题、填空题等
无法分界，选择题、填空题等
无法分界，选择题、填空题等
无法分界，选择题、填空题等
无法分界，选择题、填空题等
无法分界，选择题、填空题等
无法分界，选择题、填空题等
无法分界，选择题、填空题等
无法分界，选择题、填空题等
无法分界，选择题、填空题等
无法分界，选择题、填空题等
无法分界，选择题、填空题等
无法分界，选择题、填空题等
无法分界，选择题、填空题等
无法分界，选择题、填空题等
无法分界，选

In [380]:
1 - len(loss) / len(examination_paper_list)

0.6996904024767802

In [388]:
per_process(loss[7]["text"]).splitlines()

['1996年普通高等学校招生全国统一考试',
 '数学',
 '(理工农医类)',
 '本试卷分第Ⅰ卷(选择题)和第Ⅱ卷(非选择题)两部分.共150分,考试时间120分钟.',
 '第Ⅰ卷(选择题共65分)',
 '一、选择题:本大题共15小题;第(1)(10)题每小题4分,第(11)(15)题每小题5分,共65分,在每小题给出的四个选项中,只有一项是符合题目要求的.',
 '(1)已知全集I=N,集合A={x│x=2n,n∈N},B={x│x=4n,n∈N},则',
 '\\[Key\\] C',
 '(1)已知全集I=N,集合A={x│x=2n,n∈N},B={x│x=4n,n∈N},则',
 '\\[Key\\] C',
 '(3)若sin2x\\cos2x,则x的取值范围是',
 '\\[Key\\] D',
 '(4)复数等于',
 '\\[Key\\] B',
 '5)如果直线l、m与平面α、β、γ满足：l=β∩γ,l//α,mα和m⊥γ那么必有',
 '(A)α⊥γ且l⊥m (B)α⊥γ且m∥β',
 '(C)m∥β且l⊥m (D)α∥β且α⊥γ',
 '\\[Key\\] A',
 '(6)当，函数的',
 '(A)最大值是1，最小值是-1',
 '(B)最大值是1，最小值是-(1/2)',
 '(C)最大值是2，最小值是-2',
 '(D)最大值是2，最小值是-1',
 '\\[Key\\] D',
 '(7)椭圆的两个焦点坐标是(B)',
 '(A)(-3,5),(-3,-3) (B)(3,3,),(3,-5)',
 '(C)(1,1,),(-7,1) (D)(7,-1,),(-1,-1)',
 '(8)若，则等于',
 '\\[Key\\] A',
 '(9)将边长为a的正方形ABCD沿对角线AC折起,使得BD=a,则三棱锥D-ABC的体积为',
 '\\[Key\\] D',
 '(10)等比数列{an}的首项a1=-1，前n项的和为Sn，若，则等于',
 '\\[Key\\] B',
 '(11)椭圆的极坐标方程为，则它在短轴上的两个顶点的极坐标是',
 '\\[Key\\] C',
 '(12)等差数列{an的前m项和为30,前2m项和为100,则它的前3m项和为',
 '(A)130 (B)170 (C)210 (D)260',
 '

In [390]:
get_topic_type_with_index(per_process(loss[7]["text"]))

{5: '一、选择题:本大题共15小题;第(1)(10)题每小题4分,第(11)(15)题每小题5分,共65分,在每小题给出的四个选项中,只有一项是符合题目要求的.'}

In [389]:
get_all_option_answer_and_options(per_process(loss[7]["text"]))

{}